In [1]:
import pandas as pd
import numpy as np
from pandasql import sqldf

from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = 30
pd.get_option("display.max_rows")
#pd.set_option('display.max_rows', 100)
#import mysql.connector # pip install mysql-connector-python

#import sqlalchemy

from sqlalchemy.engine import result
from sqlalchemy import create_engine, inspect, MetaData,Table, Column, Numeric, Integer, VARCHAR
from sqlalchemy import text


import pymysql #pip install PyMySQL
pymysql.install_as_MySQLdb()


import os
from dotenv import load_dotenv

load_dotenv()  # Load environment variables from .env file

my_host = os.getenv("my_host")
my_user = os.getenv("my_user")
my_pswd = os.getenv("my_pswd")

### Create Engine: NON SPECIFIC DATABASE
* connect to MYSQL

In [2]:
# Database credentials
username = my_user
password = my_pswd
server = my_host
#database = "db_rizal"

# Establish a connection to the MySQL database
engine_stmt = f'mysql+mysqldb://{username}:{password}@{server}:3306/'#{database}'

engine = create_engine(engine_stmt)

In [3]:
inspector = inspect(engine)
database_names = inspector.get_schema_names()
print(database_names)

print("\nDatabases available:")
for db_name in database_names:
    print(f"- {db_name}")

['db_rizal', 'information_schema', 'mysql', 'performance_schema', 'sys']

Databases available:
- db_rizal
- information_schema
- mysql
- performance_schema
- sys


### SQL Query to multiple db:

In [4]:
sql_qry = """SELECT player_from,player_to FROM db_rizal.tbl_players """

In [5]:
#results = engine.execute(sql)
with engine.connect() as conn:
    result = conn.execute(text(sql_qry))

# View the records
for record in result:
    print("\n", record)


 ('Long', 'Han')

 ('Brown', 'Mariela')

 ('Fry', 'Alba')

 ('Neda', 'Cherryl')

 ('Stephen', 'Thanh')

 ('Perez', 'Eulalia')

 ('Creech', 'Thanh')

 ('Taryn', 'Hilaria')

 ('Chi', 'Renna')

 ('Long', 'Theola')

 ('Marvin', 'Theola')

 ('Rebecca', 'Danae')

 ('Ramirez', 'Han')

 ('Denise', 'Hilaria')

 ('Bennett', 'Theola')

 ('Jamal', 'Exie')

 ('Kenyatta', 'Jamal')

 ('Lavenia', 'Theola')

 ('King', 'Williams')

 ('Decker', 'Bree')

 ('Becerra', 'Nell')

 ('Alexander', 'Jamal')

 ('Jerry', 'Gertha')

 ('Keith', 'Gertha')

 ('Philip', 'Tiara')

 ('Hilaria', 'Thanh')

 ('Duarte', 'Bree')

 ('Ivory', 'Mariela')

 ('Regenia', 'Renna')

 ('Michael', 'Hilaria')

 ('Carol', 'Zina')

 ('Jeffrey', 'Renna')

 ('Mathilde', 'Theola')

 ('Karyl', 'Nell')

 ('Carver', 'Cherryl')

 ('Bailey', 'Argentina')

 ('Walker', 'Johnny')

 ('Pierson', 'Gwyn')

 ('Peter', 'Gregory')

 ('Williams', 'Zina')

 ('Edwards', 'Scottie')

 ('Carver', 'Morgan')

 ('Hall', 'Jamal')

 ('Valencia', 'Gertha')

 ('Corcora

In [6]:
### DOWNLOAD AS PANDAS

df = pd.read_sql_query(sql_qry, con=engine)
print(df.shape)
df.head()

(400, 2)


,player_from,player_to
0,Long,Han
1,Brown,Mariela
2,Fry,Alba
3,Neda,Cherryl
4,Stephen,Thanh


In [7]:
qry1 = """ 
    DROP TABLE IF EXISTS db_rizal.tmp_count;
    """

qry2 = """ 
    CREATE TABLE db_rizal.tmp_count AS
            SELECT 
            subject
            ,COUNT(1) AS FREQ
            FROM db_rizal.studentinfo
            GROUP BY subject;
    """

with engine.connect() as conn:
    conn.execute(text(qry1))
    conn.execute(text(qry2))

In [8]:
qry = """select * from db_rizal.tmp_count """

with engine.connect() as conn:
    result = conn.execute(text(qry))
    
# View the records
for record in result:
    print("\n", record)



df = pd.read_sql_query(qry, con=engine)
df


 ('Data Science', 3)

 ('Math', 3)

 ('Stat', 1)


,subject,FREQ
0,Data Science,3
1,Math,3
2,Stat,1


In [9]:
# Close the engine connection
engine.dispose()

## Create Engine: DATABASE SPECIFIC
* Connect to specific database

In [10]:
database = "db_rizal"

# Establish a connection to the MySQL database
engine_stmt = f'mysql+mysqldb://{username}:{password}@{server}:3306/{database}'


engine = create_engine(engine_stmt)

In [11]:
inspector = inspect(engine)

# Get a list of table names
table_names = inspector.get_table_names()

print("Tables in the database:")
for table_name in table_names:
    print(table_name)

Tables in the database:
employees
employees_v2
players
studentinfo
tbl_players
tbl_players_v2
tbl_test
tmp_count
tmp_freq


## CREATE TABLE:
* from csv,excel,parquet,etc:
* from code

In [12]:
### READ CSV:
csv_file_path = "players.csv"

# Read the CSV file into a pandas DataFrame
try:
    df = pd.read_csv(csv_file_path)
    print(df.head())
except FileNotFoundError:
    print(f"Error: CSV file not found at {csv_file_path}")
    exit()

  player_from player_to         label  weight
0        Long       Han         rival       8
1       Brown   Mariela  acquaintance       2
2         Fry      Alba         rival       4
3        Neda   Cherryl  acquaintance       8
4     Stephen     Thanh        friend       8


### Create New Table (from CSV):
1) directly from df dtypes


In [13]:
#1) directly from df dtypes
table_name = "tbl_players"


# CREATE TABLE (if it doesn't exist)
df.head(0).to_sql(name=table_name, con=engine, if_exists='replace', index=False)#, dtype=column_definitions)

# APPEND:  df to new tbl, Insert the data from the DataFrame into the table
df.to_sql(name=table_name, con=engine, if_exists='append', index=False)

# READ SQL TABLE
df = pd.read_sql_table(table_name, con=engine)
print(table_name,df.shape)
df.head()

tbl_players (400, 4)


,player_from,player_to,label,weight
0,Long,Han,rival,8
1,Brown,Mariela,acquaintance,2
2,Fry,Alba,rival,4
3,Neda,Cherryl,acquaintance,8
4,Stephen,Thanh,friend,8


### Create New Table (from CSV):
2) from CREATE TABLE qry

In [14]:
## 2) from CREATE TABLE qry
table_name2 = "tbl_players_v2"

# CREATE TABLE (if it doesn't exist)
qry1 = f""" DROP TABLE IF EXISTS {table_name2} """


qry2 = f"""
       CREATE TABLE {table_name2}
            (
            id INT AUTO_INCREMENT PRIMARY KEY,
            player_from varchar(255) NOT NULL,
            player_to varchar(255) NOT NULL,
            label varchar(255) NOT NULL,
            weight INT
            )
        """


with engine.connect() as conn:
    conn.execute(text(qry1))
    conn.execute(text(qry2))



# APPEND df to new tbl, Insert the data from the DataFrame into the table
df.to_sql(name=table_name2, con=engine, if_exists='append', index=False)


# READ SQL TABLE
df2 = pd.read_sql_table(table_name2, con=engine)
print(table_name2,df2.shape)
df2.head()

tbl_players_v2 (400, 5)


,id,player_from,player_to,label,weight
0,1,Long,Han,rival,8
1,2,Brown,Mariela,acquaintance,2
2,3,Fry,Alba,rival,4
3,4,Neda,Cherryl,acquaintance,8
4,5,Stephen,Thanh,friend,8


### Create New Table:
* from sql codes

In [15]:
qry1 = """DROP TABLE IF EXISTS employees"""

qry2 = """
        CREATE TABLE employees (
            id INT AUTO_INCREMENT PRIMARY KEY,
            name VARCHAR(50),
            department VARCHAR(50),
            salary DECIMAL(10, 2)
            ); 
        """

qry3 = """ 
        INSERT INTO employees (name, department, salary) 
        VALUES
            ('John Doe', 'Sales', 55000.00),
            ('Jane Smith', 'Sales', 60000.00),
            ('Jim Brown', 'Sales', 65000.00),
            ('Jake White', 'Engineering', 75000.00),
            ('Jill Green', 'Engineering', 80000.00),
            ('Jenny Black', 'Engineering', 85000.00),
            ('James Gray', 'Marketing', 50000.00),
            ('Janet Blue', 'Marketing', 52000.00),
            ('Joan Pink', 'Marketing', 54000.00);
        """

with engine.connect() as conn:
    conn.execute(text(qry1))
    conn.execute(text(qry2))
    conn.execute(text(qry3))
    conn.commit() # Commit the changes to the database


df = pd.read_sql_table('employees', con=engine)
print(df.shape)
df.head()

(9, 4)


,id,name,department,salary
0,1,John Doe,Sales,55000.0
1,2,Jane Smith,Sales,60000.0
2,3,Jim Brown,Sales,65000.0
3,4,Jake White,Engineering,75000.0
4,5,Jill Green,Engineering,80000.0


### Download table from database:

In [16]:
df = pd.read_sql_table('employees', con=engine)
print(df.shape)
df

(9, 4)


,id,name,department,salary
0,1,John Doe,Sales,55000.0
1,2,Jane Smith,Sales,60000.0
2,3,Jim Brown,Sales,65000.0
3,4,Jake White,Engineering,75000.0
4,5,Jill Green,Engineering,80000.0
5,6,Jenny Black,Engineering,85000.0
6,7,James Gray,Marketing,50000.0
7,8,Janet Blue,Marketing,52000.0
8,9,Joan Pink,Marketing,54000.0


### Download by SQL query:
* WITH CTEs
* RECURSIVE CTEs

In [17]:
### with CTEs

qry4 = """ WITH department_salaries AS (
                SELECT 
                department
                ,COUNT(1) AS n_staff
                ,SUM(salary) AS total_salary
                ,AVG(salary) AS average_salary
                FROM employees
                GROUP BY department
                ),

            capex AS (
                SELECT 
                SUM(salary) AS overall_capex
                FROM employees
            )
                
            SELECT 
            department
            ,n_staff
            ,total_salary
            ,average_salary
            #,overall_capex
            ,total_salary/overall_capex AS perc_capex
            FROM department_salaries,capex 
            
            """


df = pd.read_sql_query(qry4, con=engine)
print(df.shape)
df.head()

(3, 5)


,department,n_staff,total_salary,average_salary,perc_capex
0,Sales,3,180000.0,60000.0,0.312500
1,Engineering,3,240000.0,80000.0,0.416667
2,Marketing,3,156000.0,52000.0,0.270833


### RECURSIVE CTEs: https://learnsql.com/blog/sql-recursive-cte/
* prepare tbl: employees_v2

In [18]:
## 2) from CREATE TABLE qry
table_name = "employees_v2"
csv_file_path = table_name + '.csv'
df = pd.read_csv(csv_file_path)

# CREATE TABLE (if it doesn't exist)
qry1 = f""" DROP TABLE IF EXISTS {table_name} """

qry2 = f"""
       CREATE TABLE {table_name}
            (
            id INT PRIMARY KEY,
            first_name varchar(255) NOT NULL,
            last_name varchar(255) NOT NULL,
            boss_id INT
            )
        """
with engine.connect() as conn:
    conn.execute(text(qry1))
    conn.execute(text(qry2))

# APPEND df to new tbl, Insert the data from the DataFrame into the table
df.to_sql(name=table_name, con=engine, if_exists='append', index=False)

# READ SQL TABLE
emp2_df = pd.read_sql_table(table_name, con=engine)
emp2_df['boss_id'] = emp2_df['boss_id'].astype('Int64')
print(table_name,emp2_df.shape)
emp2_df

employees_v2 (10, 4)


,id,first_name,last_name,boss_id
0,1,Domenic,Leaver,5
1,2,Cleveland,Hewins,1
2,3,Kakalina,Atherton,8
3,4,Roxanna,Fairlie,<NA>
4,5,Hermie,Comsty,4
5,6,Pooh,Goss,8
6,7,Faulkner,Challiss,5
7,8,Bobbe,Blakeway,4
8,9,Laurene,Burchill,1
9,10,Augusta,Gosdin,8


### RECURSIVE CTEs:
* create employee hierarchy based on employee id

In [19]:
qry = """
        WITH RECURSIVE company_hierarchy AS (

        # BASE TABLE        
        
         SELECT * 
             ,0 AS hierarchy_level
             FROM employees_v2
             WHERE boss_id IS NULL

         
         UNION ALL

        
        # CONSECUTIVE TABLE
        
         SELECT  
             e.id
             ,e.first_name
             ,e.last_name
             ,e.boss_id
             ,hierarchy_level + 1
         FROM employees_v2 e , company_hierarchy ch
         WHERE e.boss_id = ch.id         
        )      

        SELECT 
            ch.id AS employee_id
            ,ch.first_name AS employee_first_name
            ,ch.last_name AS employee_last_name
            ,ch.hierarchy_level
            ,ch.boss_id AS employee_boss_id
            ,e.first_name AS boss_first_name
            ,e.last_name AS boss_last_name
            
            

         FROM company_hierarchy ch
         LEFT JOIN employees_v2 e ON e.id = ch.boss_id
         ORDER BY ch.hierarchy_level, ch.boss_id
       """


df = pd.read_sql_query(qry, con=engine)
df['employee_boss_id'] = df['employee_boss_id'].astype('Int32')
print(df.shape)
df

(10, 7)


,employee_id,employee_first_name,employee_last_name,hierarchy_level,employee_boss_id,boss_first_name,boss_last_name
0,4,Roxanna,Fairlie,0,<NA>,None,None
1,5,Hermie,Comsty,1,4,Roxanna,Fairlie
2,8,Bobbe,Blakeway,1,4,Roxanna,Fairlie
3,1,Domenic,Leaver,2,5,Hermie,Comsty
4,7,Faulkner,Challiss,2,5,Hermie,Comsty
5,3,Kakalina,Atherton,2,8,Bobbe,Blakeway
6,6,Pooh,Goss,2,8,Bobbe,Blakeway
7,10,Augusta,Gosdin,2,8,Bobbe,Blakeway
8,2,Cleveland,Hewins,3,1,Domenic,Leaver
9,9,Laurene,Burchill,3,1,Domenic,Leaver


In [21]:
# Close the engine connection
engine.dispose()